In [1]:
import os, parse3ddmp

# dump3d -d Jahedong.3d > Jahedong.dmp
dname = "/home/julian/data/madphilpointclouds/janet/Jahedong.dmp"
dmp3d = parse3ddmp.DMP3d(open(dname).readlines())



In [2]:
parse3ddmp.exportfortunnelvr

<function parse3ddmp.exportfortunnelvr(jsfile, dmp3d)>

In [3]:
dmp3d.cs

'+proj=utm +ellps=WGS84 +datum=WGS84 +units=m +zone=49 +no_defs'

In [4]:
import pyproj


In [18]:
from parse3ddmp import P3

if True:
    entrancenodes = [node  for node in dmp3d.nodes  if "ENTRANCE" in node[2]]
    svxp0 = max((node[0]  for node in entrancenodes or dmp3d.nodes), key=(lambda X: (X[2], X[0], X[1])))
    svxp0 = P3(svxp0[0], svxp0[1], 0.0)
    csrec = { "svxp0":svxp0, "title":dmp3d.title, "headdate":dmp3d.headdate.isoformat() }
    if dmp3d.cs:
        print("We have a coordinate system (cs) ", dmp3d.cs)
        import pyproj
        proj = pyproj.Proj(dmp3d.cs)
        lngp0, latp0 = proj(svxp0[0], svxp0[1], inverse=True)
        lngp0, latp0
        ddeg = 0.01
        pxn, pyn = proj(lngp0, latp0+ddeg)
        nyfac, nxfac = (pyn-svxp0[1])/ddeg, (pxn-svxp0[0])/ddeg 
        pxe, pye = proj(lngp0+ddeg, latp0)
        eyfac, exfac = (pye-svxp0[1])/ddeg, (pxe-svxp0[0])/ddeg 
        csrec["svxp0"] = P3(lngp0, latp0, svxp0[2])
        csrec.update({ "cs":dmp3d.cs, "nyfac":nyfac, "nxfac":nxfac, "eyfac":eyfac, "exfac":exfac })

    leglines = [line  for line in dmp3d.lines  if line[0] != line[1] and "SURFACE" not in line[4]]
    xsects = [xsect  for xsect in dmp3d.xsects  if len(xsect) >= 2]

    # points as triples
    stationpoints = set()
    stationpoints.update(line[0]  for line in leglines)
    stationpoints.update(line[1]  for line in leglines)
    stationpoints = list(stationpoints)
    stationpoints.sort(key=lambda X: (X[2], X[0], X[1]))

    stationpointsdict = dict((p, i)  for i, p in enumerate(stationpoints))

    def convp(p):
        if "cs" in csrec:
            px, py = proj(p[0], p[1], inverse=True)
            p = P3(px, py, p[2])
            r = p - csrec["svxp0"]
            r = P3(csrec["nxfac"]*r[1] + csrec["exfac"]*r[0], csrec["nyfac"]*r[1] + csrec["eyfac"]*r[0], r[2])
        else:
            r = p - csrec["svxp0"]
        return r

    stationpointscoords = sum((tuple(convp(stationpoint))  for stationpoint in stationpoints), ())
    stationpointsnames = [ dmp3d.nodepmap[stationpoint][0][0] if stationpoint in dmp3d.nodepmap else ".%dy"%i \
                           for i, stationpoint in enumerate(stationpoints) ]
    legsconnections = sum(((stationpointsdict[legline[0]], stationpointsdict[legline[1]])  for legline in leglines), ())
    legsconnections = sum(((stationpointsdict[legline[0]], stationpointsdict[legline[1]])  for legline in leglines), ())
    legsstyles = [ legline[3]  for legline in leglines ]

    xsectgps = [ ]
    for xsectseq in xsects:
        xsectpoints, xsectindexes = [ ], [ ]
        for xs in xsectseq:
            xsp = dmp3d.nmapnodes[xs[0]]
            if xsp in stationpointsdict:  # avoid using a tube on a surface leg
                xsi = stationpointsdict[xsp]
                xsectpoints.append(xsp)
                xsectindexes.append(xsi)
        xsectrightvecs = [ ]
        for i in range(len(xsectpoints)):
            pback = convp(xsectpoints[max(0, i-1)])
            p = convp(xsectpoints[i])
            pfore = convp(xsectpoints[min(len(xsectpoints)-1, i+1)])
            vback = P3.ZNorm(P3(p.x - pback.x, p.y - pback.y, 0.0))
            vfore = P3.ZNorm(P3(pfore.x - p.x, pfore.y - p.y, 0.0))
            xsectplanevec = P3.ZNorm(vback + vfore)
            xsectrightvecs.append(xsectplanevec[1])
            xsectrightvecs.append(-xsectplanevec[0])
        xsectrightvecs
        xsectlruds = sum((xs[1]  for xs in xsectseq), ())
        xsectlruds
        xsectgps.append({ "xsectindexes":xsectindexes, "xsectrightvecs":xsectrightvecs, "xsectlruds":xsectlruds })
    xsectgps

    jrec = { "stationpointscoords": stationpointscoords,
             "stationpointsnames":  stationpointsnames, 
             "legsconnections":     legsconnections,
             "legsstyles":          legsstyles, 
             "xsectgps":            xsectgps
           }
    jrec.update(csrec)
    #return jrec

We have a coordinate system (cs)  +proj=utm +ellps=WGS84 +datum=WGS84 +units=m +zone=49 +no_defs


{'svxp0': P3(110.19522848557897, 25.25362198564808, 0.0),
 'title': 'Jahedong',
 'headdate': '2021-08-15T20:41:18',
 'cs': '+proj=utm +ellps=WGS84 +datum=WGS84 +units=m +zone=49 +no_defs',
 'nyfac': 110739.43562903441,
 'nxfac': 663.7564990960527,
 'eyfac': -599.7669206000865,
 'exfac': 100707.86398361088}

In [37]:
print(convp(P3(418824.53, 2793281.64, 166.03)))
print(convp(P3(418824.53+1, 2793281.64, 166.03)))
print(convp(P3(418824.53, 2793281.64+1, 166.03)))


P3(-131.99994022565278, 7.374440818086435, 166.03)
P3(-130.99994077703764, 7.374487799938519, 166.03)
P3(-131.99994889061097, 8.374440985122428, 166.03)
(110.19391737219604, 25.253681477358008)
(110.19392730154448, 25.253681531559806)


In [23]:
entrancenodes

[(P3(418956.53, 2793274.26, 189.95), 'jahedong.74', {'ENTRANCE'}),
 (P3(418923.29, 2793409.73, 181.27), 'jahedong.27', {'ENTRANCE'}),
 (P3(418751.13, 2793245.94, 182.34), 'jahedong.2', {'ENTRANCE'})]

In [29]:
110739*360/1000

39866.04

In [24]:
svxp0

P3(418956.53, 2793274.26, 0.0)

In [8]:
jrec = exportfortunnelvr(None, dmp3d)

We have a coordinate system (cs)  +proj=utm +ellps=WGS84 +datum=WGS84 +units=m +zone=49 +no_defs


In [10]:
jrec.keys()

dict_keys(['stationpointscoords', 'stationpointsnames', 'legsconnections', 'legsstyles', 'xsectgps', 'svxp0', 'title', 'headdate', 'cs', 'nyfac', 'nxfac', 'eyfac', 'exfac'])

In [12]:
jrec["svxp0"]

P3(110.19522848557897, 25.25362198564808, 0.0)

In [15]:
jrec["eyfac"]

-599.7669206000865

In [39]:
# convert P4180072.JPG -resize 600   -fill '#0005' -draw 'rectangle 0,0,214,45' -fill white -pointsize 42  -annotate +10+41 'Station 1' P4180072-small3.JPG

# script for over-drawing the survey station names onto the images and shrinking them
import os, subprocess, re
fdir = "/home/julian/Pictures/skirwith"
for fname in os.listdir(fdir):
    m = re.match("S(\d+)", fname)
    mfname, extname = os.path.splitext(fname)
    if m:
        mfname, extname = os.path.splitext(fname)
        fnamein = os.path.join(fdir, fname)
        fnameout = os.path.join(fdir, "m"+fname.lower())
        msg = mfname.lower()
        print(fnamein, fnameout, msg)
        pargs = ["convert", fnamein, "-resize", "600", "-fill", '#0005', "-draw", 'rectangle 0,0,144,45',
                 "-fill", "white", "-pointsize", "42", "-annotate", "+10+41", msg, fnameout]
        x = subprocess.run(pargs, capture_output=True)


/home/julian/Pictures/skirwith/S55.JPG /home/julian/Pictures/skirwith/ms55.jpg s55
/home/julian/Pictures/skirwith/S56.JPG /home/julian/Pictures/skirwith/ms56.jpg s56
/home/julian/Pictures/skirwith/S51.JPG /home/julian/Pictures/skirwith/ms51.jpg s51
/home/julian/Pictures/skirwith/S48.JPG /home/julian/Pictures/skirwith/ms48.jpg s48
/home/julian/Pictures/skirwith/S33.JPG /home/julian/Pictures/skirwith/ms33.jpg s33
/home/julian/Pictures/skirwith/S4.JPG /home/julian/Pictures/skirwith/ms4.jpg s4
/home/julian/Pictures/skirwith/S83.JPG /home/julian/Pictures/skirwith/ms83.jpg s83
/home/julian/Pictures/skirwith/S1.JPG /home/julian/Pictures/skirwith/ms1.jpg s1
/home/julian/Pictures/skirwith/S25.JPG /home/julian/Pictures/skirwith/ms25.jpg s25
/home/julian/Pictures/skirwith/S70a.JPG /home/julian/Pictures/skirwith/ms70a.jpg s70a
/home/julian/Pictures/skirwith/S40.JPG /home/julian/Pictures/skirwith/ms40.jpg s40
/home/julian/Pictures/skirwith/S11.JPG /home/julian/Pictures/skirwith/ms11.jpg s11
/home/j

In [2]:
import os
print(os.listdir())

['dukest1resurvey2009.json', 'smallirebysave.res', 'dukest1resurvey2009.tscn', '0_917647_0_917647_0_917647_0_000000.material', 'dukest1resurvey2009.3d', 'parse3D_work.ipynb', 'picturepalace2.gltf', 'LambTrap-drawnup-1.png', 'LargePot', '0_980392_0_713725_0_003922_0_000000.material', 'dukest1resurvey2009.gltf', 'scanimagefloor.material', 'skirwith', 'LambTrap-drawnup-1.png.import', '0_498039_0_498039_0_498039_0_000000.material', 'picturehouse.glb.import', '0_615686_0_811765_0_929412_0_000000.material', 'picturepalace.gltf', '.ipynb_checkpoints', 'wingform', '0_972549_0_529412_0_003922_0_000000.material', '0_647059_0_647059_0_647059_0_000000.material', '__pycache__', '0_729412_0_250980_0_105882_0_000000.material', 'picturepalace.gltf.import', 'Ireby', 'picturehouse.glb', 'parse3ddmp.py', 'dukest1resurvey2009.gltf.import', 'DukeStResurvey-drawnup-p3.jpg.import', 'picturepalace2.gltf.import', 'notes.txt', 'convertdmptotunnelvrjson.py', 'DukeStResurvey-drawnup-p3.jpg', '0_372549_0_654902_0_

In [7]:
from parse3ddmp import DMP3d

fname = "skirwith/skirwith-cave.3d"
dump3dexe = 'dump3d'

dstream = os.popen("%s -d %s" % (dump3dexe, fname))
lines = dstream.readlines()
dmp3d = DMP3d(lines)


In [8]:
dmp3d.cs

'+init=epsg:27700 +no_defs'

In [14]:
import pyproj
proj = pyproj.Proj(dmp3d.cs)
proj(370971.19, 473746.38, inverse=True)


(-2.4445880673413165, 54.15853521413708)

In [11]:
leglines = [line  for line in dmp3d.lines  if line[0] != line[1] and "SURFACE" not in line[4]]
len(leglines)
leg = leglines[0]
leg

(P3(370971.19, 473746.38, 259.43),
 P3(370969.28, 473746.58, 259.1),
 'skirwith-cave.skirwith-upper-entrance-1',
 'NORMAL',
 [],
 datetime.datetime(2021, 4, 12, 0, 0))

In [12]:
dir(dmp3d)

['GetXSectionFacts',
 'GetXSectionQuadHedron',
 'GetXSectionQuadHedronP',
 'PassagesLength',
 'PassagesVolume',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cs',
 'datenumeric',
 'headdate',
 'lines',
 'nmapnodes',
 'nodepmap',
 'nodes',
 'printreport',
 'title',
 'xsects']

In [52]:
from parse3ddmp import DMP3d
import os

dump3dexe = '"C:\\Program Files (x86)\\Survex\\dump3d.exe"'
fname = "dukest1resurvey2009.3d"
fjson = "dukest1resurvey2009.json"
fname = "cusseypot\\Cussey_Pot_Master.3d"
fjson = "cusseypot\\Cussey_Pot_Master.json"
fname = "cusseypot\\skirwith-cave.3d"


dump3dexe = 'dump3d'
fname = "/home/julian/Downloads/fountainsfell.3d"
fjson = "test.json"

fname = "/home/julian/.local/share/godot/app_userdata/tunnelvr_v0.5/nonimagewebpages/WSBab3dd7b423a5065f152a7ment.3d"
fjson = "/home/julian/.local/share/godot/app_userdata/tunnelvr_v0.5/executingfeatures/fcentreline.json"


dstream = os.popen("%s -d %s" % (dump3dexe, fname))
lines = dstream.readlines()
dmp3d = DMP3d(lines)

"""
    self.nodepmap = { }   # { p: ([names], [flagsconsolidated] }  
    self.nodes = [ ]      # [ (p, name, [flags]) ]   (not strictly necessary, except to get entrance name before flagconsolidation)
    self.lines = [ ]      # [ (prevp, p, name, style, [flags], date) ]
    self.xsects = [ [ ] ] # [ [ (name, lrud) ] ]
    self.nmapnodes = { }  # { name: p } (derived from nodepmap, needed for xsects)
""";

In [53]:
from parse3ddmp import P3
import json

entrancenodes = [node  for node in dmp3d.nodes  if "ENTRANCE" in node[2]]
svxp0 = max((node[0]  for node in entrancenodes or dmp3d.nodes), key=(lambda X: (X[2], X[0], X[1])))
csrec = { "svxp0":svxp0, "title":dmp3d.title, "headdate":dmp3d.headdate.isoformat() }
print(csrec)
if dmp3d.cs:
    import pyproj
    proj = pyproj.Proj(dmp3d.cs)
    lngp0, latp0 = proj(svxp0[0], svxp0[1], inverse=True)
    lngp0, latp0
    ddeg = 0.01
    pxn, pyn = proj(lngp0, latp0+ddeg)
    nyfac, nxfac = (pyn-svxp0[1])/ddeg, (pxn-svxp0[0])/ddeg 
    pxe, pye = proj(lngp0+ddeg, latp0)
    eyfac, exfac = (pye-svxp0[1])/ddeg, (pxe-svxp0[0])/ddeg 
    csrec["svxp0"] = P3(lngp0, latp0, svxp0[2])
    csrec.update({ "cs":dmp3d.cs, "nyfac":nyfac, "nxfac":nxfac, "eyfac":eyfac, "exfac":exfac })
csrec

{'svxp0': P3(71831.0, 74646.0, 305.0), 'title': 'WhiteScarCave.3d', 'headdate': '2021-05-17T08:54:15'}


{'svxp0': P3(-2.4315042646496923, 54.16666908118276, 305.0),
 'title': 'WhiteScarCave.3d',
 'headdate': '2021-05-17T08:54:15',
 'cs': '+proj=tmerc +lat_0=49 +lon_0=-2 +k=0.9996012717 +x_0=100000 +y_0=-500000 +ellps=airy +datum=OSGB36 +units=m +no_defs',
 'nyfac': 111252.86369665992,
 'nxfac': 679.33181966946,
 'eyfac': -393.97198546212167,
 'exfac': 65280.55680047546}

In [54]:
def convp(p):
    r = p - csrec["svxp0"]
    if "cs" in csrec:
        r = P3(csrec["nxfac"]*r[1] + csrec["exfac"]*r[0], csrec["nyfac"]*r[1] + csrec["eyfac"]*r[0], r[2])
    return r
convp(P3(-2.221681512328065, 54.127077652039944, 457.9))

P3(13670.450383361172, -4487.324156309066, 152.89999999999998)

In [73]:
set(stationpoints).difference(set(dmp3d.nodepmap.keys()))
x = min(stationpoints)
#dmp3d.nodepmap[x]
min(dmp3d.nodepmap.keys())
len(dmp3d.nodepmap), len(set(stationpoints))
leglines
#len(stationpointsdict)
#dmp3d.nodepmap

[(P3(71831.0, 74646.0, 305.0),
  P3(71832.02, 74646.16, 321.14),
  'WhiteScarIndex.NorthernLine',
  '',
  ['SPLAY'],
  datetime.datetime(2019, 7, 3, 0, 0)),
 (P3(71831.0, 74646.0, 305.0),
  P3(71831.58, 74646.04, 308.77),
  'WhiteScarIndex.NorthernLine',
  '',
  ['SPLAY'],
  datetime.datetime(2019, 7, 3, 0, 0)),
 (P3(71831.0, 74646.0, 305.0),
  P3(71831.21, 74646.01, 306.13),
  'WhiteScarIndex.NorthernLine',
  '',
  ['SPLAY'],
  datetime.datetime(2019, 7, 3, 0, 0)),
 (P3(71831.0, 74646.0, 305.0),
  P3(71831.21, 74645.81, 303.85),
  'WhiteScarIndex.NorthernLine',
  '',
  [],
  datetime.datetime(2019, 7, 3, 0, 0)),
 (P3(71831.21, 74645.81, 303.85),
  P3(71832.45, 74645.32, 303.85),
  'WhiteScarIndex.NorthernLine',
  '',
  ['SPLAY'],
  datetime.datetime(2019, 7, 3, 0, 0)),
 (P3(71831.21, 74645.81, 303.85),
  P3(71831.3, 74644.41, 303.43),
  'WhiteScarIndex.NorthernLine',
  '',
  ['SPLAY'],
  datetime.datetime(2019, 7, 3, 0, 0)),
 (P3(71831.21, 74645.81, 303.85),
  P3(71831.53, 74644.42, 3

In [74]:
from parse3ddmp import P3
import json

entrancenodes = [node  for node in dmp3d.nodes  if "ENTRANCE" in node[2]]
svxp0 = max((node[0]  for node in entrancenodes or dmp3d.nodes), key=(lambda X: (X[2], X[0], X[1])))
csrec = { "svxp0":svxp0, "title":dmp3d.title, "headdate":dmp3d.headdate.isoformat() }
if dmp3d.cs:
    import pyproj
    proj = pyproj.Proj(dmp3d.cs)
    lngp0, latp0 = proj(svxp0[0], svxp0[1], inverse=True)
    lngp0, latp0
    ddeg = 0.01
    pxn, pyn = proj(lngp0, latp0+ddeg)
    nyfac, nxfac = (pyn-svxp0[1])/ddeg, (pxn-svxp0[0])/ddeg 
    pxe, pye = proj(lngp0+ddeg, latp0)
    eyfac, exfac = (pye-svxp0[1])/ddeg, (pxe-svxp0[0])/ddeg 
    csrec["svxp0"] = P3(lngp0, latp0, svxp0[2])
    csrec.update({ "cs":dmp3d.cs, "nyfac":nyfac, "nxfac":nxfac, "eyfac":eyfac, "exfac":exfac })

leglines = [line  for line in dmp3d.lines  if line[0] != line[1] and "SURFACE" not in line[4]]
xsects = [xsect  for xsect in dmp3d.xsects  if len(xsect) >= 2]

# points as triples
stationpoints = set()
stationpoints.update(line[0]  for line in leglines)
stationpoints.update(line[1]  for line in leglines)
stationpoints = list(stationpoints)
stationpoints.sort(key=lambda X: (X[2], X[0], X[1]))

stationpointsdict = dict((p, i)  for i, p in enumerate(stationpoints))

def convp(p):
    if "cs" in csrec:
        px, py = proj(p[0], p[1], inverse=True)
        p = P3(px, py, p[2])
        r = p - csrec["svxp0"]
        r = P3(csrec["nxfac"]*r[1] + csrec["exfac"]*r[0], csrec["nyfac"]*r[1] + csrec["eyfac"]*r[0], r[2])
    else:
        r = p - csrec["svxp0"]
    return r

stationpointscoords = sum((tuple(convp(stationpoint))  for stationpoint in stationpoints), ())
stationpointsnames = [ dmp3d.nodepmap[stationpoint][0][0] if stationpoint in dmp3d.nodepmap else ".%ds"%i \
                       for i, stationpoint in enumerate(stationpoints) ]
legsconnections = sum(((stationpointsdict[legline[0]], stationpointsdict[legline[1]])  for legline in leglines), ())
legsconnections = sum(((stationpointsdict[legline[0]], stationpointsdict[legline[1]])  for legline in leglines), ())
legsstyles = [ legline[3]  for legline in leglines ]

xsectgps = [ ]
for xsectseq in xsects:
    xsectpoints = [dmp3d.nmapnodes[xs[0]]  for xs in xsectseq]
    xsectindexes = [ stationpointsdict[xsectpoint]  for xsectpoint in xsectpoints ]
    xsectrightvecs = [ ]
    for i in range(len(xsectpoints)):
        pback = convp(xsectpoints[max(0, i-1)])
        p = convp(xsectpoints[i])
        pfore = convp(xsectpoints[min(len(xsectpoints)-1, i+1)])
        vback = P3.ZNorm(P3(p.x - pback.x, p.y - pback.y, 0.0))
        vfore = P3.ZNorm(P3(pfore.x - p.x, pfore.y - p.y, 0.0))
        xsectplanevec = P3.ZNorm(vback + vfore)
        xsectrightvecs.append(xsectplanevec[1])
        xsectrightvecs.append(-xsectplanevec[0])
    xsectrightvecs
    xsectlruds = sum((xs[1]  for xs in xsectseq), ())
    xsectlruds
    xsectgps.append({ "xsectindexes":xsectindexes, "xsectrightvecs":xsectrightvecs, "xsectlruds":xsectlruds })
xsectgps

jrec = { "stationpointscoords": stationpointscoords,
         "stationpointsnames":  stationpointsnames, 
         "legsconnections":     legsconnections,
         "legsstyles":          legsstyles, 
         "xsectgps":            xsectgps
       }
jrec.update(csrec)
print(" ".join("%s:%d"%(k, len(v))  for k, v in jrec.items()  if type(v) in [tuple, list]))
open(fjson, "w").write(json.dumps(jrec))

stationpointscoords:2667 stationpointsnames:889 legsconnections:1776 legsstyles:888 xsectgps:0


76537

In [51]:
def round_floats(o):
    if isinstance(o, float):
        return round(o, 5)
    if isinstance(o, dict):
        return {k: round_floats(v) for k, v in o.items()}
    if isinstance(o, (list, tuple)):
        return [round_floats(x) for x in o]
    return o
json.dumps(round_floats(jrec))


'{"stationpointscoords": [-1389.20251, 1732.93092, -198.16, -1391.12345, 1733.6702, -194.81, -1389.20251, 1732.93092, -194.81, -1399.50862, 1740.30705, -193.29, -1397.7875, 1738.7577, -193.29, -1396.1267, 1738.18832, -193.29, -1393.63518, 1736.24926, -193.29, -1392.21415, 1734.61979, -193.29, -1391.12345, 1733.6702, -193.29, -1400.5194, 1741.60667, -188.89, -1399.50862, 1740.30705, -188.89, -1267.98041, 912.88589, -184.13, -1260.81829, 910.28836, -183.47, -1404.75215, 1745.41508, -183.09, -1402.48065, 1743.30593, -183.09, -1400.5194, 1741.60667, -183.09, -1275.44191, 913.0133, -182.55, -1309.68643, 905.2913, -181.7, -1290.35314, 906.98811, -181.7, -1063.12547, 719.7823, -181.61, -801.11564, 666.82379, -181.58, -796.75506, 667.12485, -181.58, -1076.46943, 727.86826, -181.51, -1062.24663, 725.37256, -181.51, -1056.66013, 700.87426, -181.48, -1058.30315, 712.90375, -181.46, -819.73026, 677.76919, -181.46, -814.09812, 670.31059, -181.46, -804.51612, 666.82295, -181.46, -1309.38146, 888.021

In [ ]:
def round_floats(o):
    if isinstance(o, float):
        return round(o, 5)
    if isinstance(o, dict):
        return {k: round_floats(v) for k, v in o.items()}
    if isinstance(o, (list, tuple)):
        return [round_floats(x) for x in o]
    return o
json.dumps(round_floats(data))

